# Mission - Classifiez automatiquement des informations
# Partie 1 - Identifiez les causes d'attrition au sein d'une ESN

Vous êtes mandaté en tant que Consultant Data Scientist par le département RH de votre client. Il s’agit de l'ESN TechNova Partners, spécialisée dans le conseil en transformation digitale et la vente d’applications en SaaS.
 
Ils font face à un turnover plus élevé que d'habitude et ils souhaitent identifier les causes racines potentielles derrière ces démissions.

## Étape 2 - Préparez la donnée pour la modélisation

#### Importation des librairies

In [1]:
# Librairies "classiques"
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
from IPython.display import Image, display

# Librairies scikit-learn
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV, 
    cross_validate,
)
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error 
from sklearn.inspection import permutation_importance

#Preprocess
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

#Modèles
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

### On reprend les éléments du précédent notebook

In [2]:
donnees_features = pd.read_csv("Data/Projet_4_etape1_clean.csv")

#### Petit check

In [3]:
donnees_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 20 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   id_employee                               1470 non-null   int64 
 1   age                                       1470 non-null   int64 
 2   genre                                     1470 non-null   object
 3   revenu_mensuel                            1470 non-null   int64 
 4   statut_marital                            1470 non-null   object
 5   poste                                     1470 non-null   object
 6   nombre_experiences_precedentes            1470 non-null   int64 
 7   annee_experience_totale                   1470 non-null   int64 
 8   annees_dans_l_entreprise                  1470 non-null   int64 
 9   satisfaction_employee_environnement       1470 non-null   int64 
 10  satisfaction_employee_nature_travail      1470 n

* Petit récapitulatif

In [5]:
shape_donnees_features = donnees_features.shape
print("Nous avons", shape_donnees_features[0],"lignes et",shape_donnees_features[1],"colonnes avant transformation")

Nous avons 1470 lignes et 20 colonnes avant transformation


#### On vérifie l'importation

In [6]:
donnees_features.head()

,id_employee,age,genre,revenu_mensuel,statut_marital,poste,nombre_experiences_precedentes,annee_experience_totale,annees_dans_l_entreprise,satisfaction_employee_environnement,satisfaction_employee_nature_travail,heure_supplementaires,augmentation_salaire_precedente_pourcent,a_quitte_l_entreprise,nombre_participation_pee,nb_formations_suivies,distance_domicile_travail,niveau_education,frequence_deplacement,annees_depuis_la_derniere_promotion
0,1,41,F,5993,Celibataire,CadreCommercial,8,8,6,2,4,Oui,11,Oui,0,0,1,2,Occasionnel,0
1,2,49,M,5130,Marie,AssistantdeDirection,1,10,10,3,2,Non,23,Non,1,3,8,1,Frequent,1
2,4,37,M,2090,Celibataire,Consultant,6,7,0,4,3,Oui,15,Oui,0,3,2,2,Occasionnel,0
3,5,33,F,2909,Marie,AssistantdeDirection,1,8,8,4,3,Oui,11,Non,0,3,3,4,Frequent,3
4,7,27,M,3468,Marie,Consultant,9,6,2,1,2,Non,12,Non,1,3,2,1,Occasionnel,2


#### Regardons la distribution de notre variable continue revenu_mensuel

In [7]:
fig13 = px.histogram(
    donnees_features, x='revenu_mensuel', color ='a_quitte_l_entreprise'
)
fig13.show()
pio.write_image(fig13,"Graph/graph_etape_2/Histo_Cible_cible_enviro.png")

* La distribution du revenu_mensuel n'est pas linéaire, on voit un étalement sur la droite. L'explication peut être simplement un écart de revenu conséquent avec les différents types de poste. Regardons de plus près avec un boxplot de cette variable.

In [8]:
fig14 = px.box(
        donnees_features,
        x='revenu_mensuel',
        title=f"Boxplot interactif - Cible et revenu",
        hover_data = "id_employee"
     
    )
fig14.update_layout(height=500, width=600)
fig14.show()
pio.write_image(fig14, "Graph/graph_etape_2/Boxplot_cible_revenu_mensuel.png")

* On remarque quelques valeurs extrêmes, elles semblent normales avec les petites analyses que nous avons réalisé dans l'étape précédente. On peut le confirmer en regardant via une méthode statistique plus poussée.

In [22]:
fig13 = px.histogram(
    donnees_features, x='age', color ='a_quitte_l_entreprise'
)
fig13.show()


#### Nous ne pouvons pas utiliser ici la méthode du z-score car nous n'avons pas une distribution normale. Regardons avec la méthode IQR.

L'écart interquartile (IQR) est une mesure de la dispersion, basée sur la division de l'ensemble de données en quartiles. Les quartiles divisent un ensemble de données classé en quatre parties égales.
Ici on va appliquer un calcul sur l'IQR, limite haute et basse pour identifier des valeurs abrérrantes/extrêmes.
* inférieures à Q1 - 1,5 * IQR 
* ou supérieures à Q3 + 1,5 * IQR 

In [9]:
# J'applique la méthode à ma variable
iqr = np.quantile(donnees_features['revenu_mensuel'], q=[0.25, 0.75])
limite_basse = iqr[0] - 1.5*(iqr[1]-iqr[0])
limite_haute = iqr[1] + 1.5*(iqr[1]-iqr[0])

In [10]:
print("La limite haute est de",limite_haute,"et la limite basse est de",limite_basse)

La limite haute est de 16581.0 et la limite basse est de -5291.0


#### Recherchons quels sont les postes avec de si gros salaires

In [11]:
(donnees_features['revenu_mensuel']>=limite_haute).value_counts()

revenu_mensuel
False    1356
True      114
Name: count, dtype: int64

In [12]:
donnees_features.loc[donnees_features['revenu_mensuel']>=limite_haute,'poste'].unique()

array(['SeniorManager', 'DirecteurTechnique'], dtype=object)

* Rien d'étonnant ici, les salaires au-dessus de 16 581 sont des directeurs, nous pouvons les garder dans notre jeu.
* Pour la limite basse, pas besoin de recherches complémentaires car nous n'avons de salaire négatif dans notre DataFrame.

Analyse globale (pas distinction de Oui et Non) les autres variables quantitatives (à partir des éléments de l'étape 1) :
* age : Nous asvon vu dans l'étape précédente que les âges allaient de 18 à 60 ans : rien d'anormal ici
* annee_experience_totale : de 0 année d'expérience (entre le début et la première année d'expérience) à 40 années: rien d'anormal ici
* distance_domicile_travail : de 1km à 29km de distance domicile travail : rien d'anormal

### Ajout de nouvelles variables

#### Reprenons notre DataFrame pour regarder les éléments que nous pouvons créer.

In [13]:
donnees_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 20 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   id_employee                               1470 non-null   int64 
 1   age                                       1470 non-null   int64 
 2   genre                                     1470 non-null   object
 3   revenu_mensuel                            1470 non-null   int64 
 4   statut_marital                            1470 non-null   object
 5   poste                                     1470 non-null   object
 6   nombre_experiences_precedentes            1470 non-null   int64 
 7   annee_experience_totale                   1470 non-null   int64 
 8   annees_dans_l_entreprise                  1470 non-null   int64 
 9   satisfaction_employee_environnement       1470 non-null   int64 
 10  satisfaction_employee_nature_travail      1470 n

#### Les features que nous voulons créer :
##### Création de calculs et ratios
Avec les notions d'années :
* ratio_stabilite = annees_dans_l_entreprise / (annee_experience_totale+1)
* experience_externe = annee_experience_totale - annees_dans_l_entreprise

Avec les sondages :
* score_satisfaction = (satisfaction_employee_environnement + satisfaction_employee_nature_travail)/2

Avec le % d'augmentation :
* augmentation_par_formation = augmentation_salaire_precedente_pourcent / (nb_formations_suivies+1)
* augmentation_par_anciennete = augmentation_salaire_precedente_pourcent / (annees_dans_l_entreprise+1)
* augmentation_relative_salaire = (augmentation_salaire_precedente_pourcent/100) * revenu_mensuel

Avec le nombre_participation_pee :
* pee_par_anciennete = nombre_participation_pee / (annees_dans_l_entreprise+1)

##### Création d'indicateurs

* Ajout de tranche d'âges
* Catégorisation de l'expérience totale : junior, senior etc
* Catégorisation de annees_dans_l_entreprise : salarié récent, ancien etc
* Catégorisation nombre_participation_pee : Oui ou Non
* Catégorisation nb_formations_suivies : Oui ou Non
* Ajout de tranches de distance_domicile_travail (proche / moyen / loin)
* Ajout de tranches annees_depuis_la_derniere_promotion

#### A gérer plus tard avec l'encodage

* genre
* statut_marital
* poste
* niveau_education
* frequence_deplacement
* satisfaction_employee_environnement
* satisfaction_employee_nature_travail
* heure_supplementaires


### Création des calculs et ratios

#### Avec les notions d'années

In [15]:
donnees_features['ratio_stabilite'] = donnees_features['annees_dans_l_entreprise'] / (donnees_features['annee_experience_totale']+1)
donnees_features['experience_externe'] = donnees_features['annee_experience_totale'] - donnees_features['annees_dans_l_entreprise']

* ratio_stabilite : plus un score est elevé, plus le salarié est resté longtemps dans l'entreeprise par rapport à sa carrière totale (fidélité à l'entreprise)
* experience_externe : nombre d'années dans une autre entreprise

* Petit récapitulatif

In [16]:
shape_donnees_features_un = donnees_features.shape
print("Nous avons", shape_donnees_features[0],"lignes et",shape_donnees_features[1],"colonnes avant transformation")
print("Nous avons", shape_donnees_features_un[0],"lignes et",shape_donnees_features_un[1],"colonnes après ajout de 2 nouvelles variables (annee_experience_totale & annees_dans_l_entreprise)")

Nous avons 1470 lignes et 20 colonnes avant transformation
Nous avons 1470 lignes et 22 colonnes après ajout de 2 nouvelles variables (annee_experience_totale & annees_dans_l_entreprise)


#### Avec les sondages

In [17]:
donnees_features['score_satisfaction'] = (donnees_features['satisfaction_employee_environnement'] + donnees_features['satisfaction_employee_nature_travail'])/2

* Petit check

In [20]:
donnees_features[['score_satisfaction','satisfaction_employee_environnement','satisfaction_employee_nature_travail']].head()

,score_satisfaction,satisfaction_employee_environnement,satisfaction_employee_nature_travail
0,3.0,2,4
1,2.5,3,2
2,3.5,4,3
3,3.5,4,3
4,1.5,1,2


In [21]:
donnees_features['score_satisfaction'].unique()

array([3. , 2.5, 3.5, 1.5, 4. , 2. , 1. ])

* Petit récapitulatif

In [18]:
shape_donnees_features_deux = donnees_features.shape
print("Nous avons", shape_donnees_features[0],"lignes et",shape_donnees_features[1],"colonnes avant transformation")
print("Nous avons", shape_donnees_features_un[0],"lignes et",shape_donnees_features_un[1],"colonnes après ajout de 2 nouvelles variables (annee_experience_totale & annees_dans_l_entreprise)")
print("Nous avons", shape_donnees_features_deux[0],"lignes et",shape_donnees_features_deux[1],"colonnes après ajout d'une variable sondage global")

Nous avons 1470 lignes et 20 colonnes avant transformation
Nous avons 1470 lignes et 22 colonnes après ajout de 2 nouvelles variables (annee_experience_totale & annees_dans_l_entreprise)
Nous avons 1470 lignes et 23 colonnes après ajout d'une variable sondage global


#### Avec le % d'augmentation :

In [25]:
donnees_features['augmentation_par_formation'] = donnees_features['augmentation_salaire_precedente_pourcent'] / (donnees_features['nb_formations_suivies']+1)
donnees_features['augmentation_par_anciennete'] = donnees_features['augmentation_salaire_precedente_pourcent'] / (donnees_features['annees_dans_l_entreprise']+1)
donnees_features['augmentation_relative_salaire'] = (donnees_features['augmentation_salaire_precedente_pourcent']/100) * donnees_features['revenu_mensuel']

In [30]:
donnees_features[
    ['augmentation_par_formation','augmentation_par_anciennete','augmentation_relative_salaire',
    'revenu_mensuel','augmentation_salaire_precedente_pourcent','annees_dans_l_entreprise','nb_formations_suivies']
    
].head(15)

,augmentation_par_formation,augmentation_par_anciennete,augmentation_relative_salaire,revenu_mensuel,augmentation_salaire_precedente_pourcent,annees_dans_l_entreprise,nb_formations_suivies
0,11.000000,1.571429,659.23,5993,11,6,0
1,5.750000,2.090909,1179.90,5130,23,10,3
2,3.750000,15.000000,313.50,2090,15,0,3
3,2.750000,1.222222,319.99,2909,11,8,3
4,3.000000,4.000000,416.16,3468,12,2,3
5,4.333333,1.625000,398.84,3068,13,7,2
6,5.000000,10.000000,534.00,2670,20,1,3
7,7.333333,11.000000,592.46,2693,22,1,2
8,7.000000,2.100000,2000.46,9526,21,9,2
9,3.250000,1.625000,680.81,5237,13,7,3
